# Aggregate Summary of Statistics and Make a Baseline Table

This notebook aggregates summary of statistics of all infection-related hospital admission data and processes them to make an aggregated baseline table.

In [1]:
import pandas as pd
import numpy as np

## 0- functions and read data

In [2]:
summary_table_lrti_incdt = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_incdt.csv')
summary_table_lrti_prevl = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_prevl.csv')
summary_table_urti_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_incdt.csv')
summary_table_urti_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_prevl.csv')
summary_table_uti_incdt = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_incdt.csv')
summary_table_uti_prevl = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_prevl.csv')
summary_table_sinusitis_incdt = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_incdt.csv')
summary_table_sinusitis_prevl = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_prevl.csv')
summary_table_otmedia_incdt = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_incdt.csv')
summary_table_otmedia_prevl = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_prevl.csv')
summary_table_ot_externa_incdt = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_incdt.csv')
summary_table_ot_externa_prevl = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_prevl.csv')
summary_table_urti_combined_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_incdt.csv')
summary_table_urti_combined_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_prevl.csv')
summary_table_cough_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_incdt.csv')
summary_table_cough_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_prevl.csv')
summary_table_cough_cold_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_incdt.csv')
summary_table_cough_cold_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_prevl.csv')
summary_table_throat_incdt = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_incdt.csv')
summary_table_throat_prevl = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_prevl.csv')

In [3]:
print(summary_table_throat_prevl['variable'].tolist())

['antibacterial_brit', 'throat_ab_date', 'age_cat_15_24', 'age_cat_25_34', 'age_cat_35_44', 'age_cat_45_54', 'age_cat_55_64', 'age_cat_65_74', 'age_cat_75_more', 'sex_female', 'sex_male', 'CCI_cat_high', 'CCI_cat_low', 'CCI_cat_medium', 'CCI_cat_very_high', 'CCI_cat_very_low', 'flu_vaccine_no', 'flu_vaccine_yes', 'bmi_cat_healthy_weight', 'bmi_cat_obese', 'bmi_cat_overweight', 'bmi_cat_underweight', 'bmi_cat_unknown', 'region_east', 'region_east_midlands', 'region_london', 'region_north_east', 'region_north_west', 'region_south_east', 'region_south_west', 'region_west_midlands', 'region_yorkshire', 'imd_affluent', 'imd_medium', 'imd_unaffluent', 'imd_unknown', 'imd_very_affluent', 'imd_very_unaffluent', 'ethnicity_asian', 'ethnicity_other', 'ethnicity_unknown', 'ethnicity_white', 'smoking_ex_smoker', 'smoking_never_smoked', 'smoking_smoker', 'smoking_unknown', 'season_autumn', 'season_spring', 'season_summer', 'season_winter', 'period_during_pandemic', 'period_post_2nd_lockdown', 'peri

In [4]:
# summary_table_throat_prevl

In [5]:
def data_process(data):
    #replace ab_date with ab_binary in variable column
    data.loc[data['variable'].str.contains('_ab_date'), 'variable'] = 'ab_binary'
    #replace SMs with 5 in data
    for row in range(0, len(data)):
        if data['count'][row] == 'SM':
            data['count'][row] = 5
    #conver counts into int
    data['count_int'] = data['count'].astype(int)#.astype(str).replace('\.0', '', regex=True).astype(int)
    #convert means into percentages with two decimals
    data['mean_perc'] = data['mean']*100
    data['mean_perc'] = np.round(data['mean_perc'], decimals=2)
    #custom order of variables to fit tables of the manuscript
    data['variable'] = pd.Categorical(data['variable'], ['age_cat_15_24', 'age_cat_25_34', 'age_cat_35_44', 'age_cat_45_54', 'age_cat_55_64', 'age_cat_65_74', 'age_cat_75_more', 'sex_female', 'sex_male', 'CCI_cat_high', 'CCI_cat_low', 'CCI_cat_medium', 'CCI_cat_very_high', 'CCI_cat_very_low', 'flu_vaccine_no', 'flu_vaccine_yes', 'bmi_cat_healthy_weight', 'bmi_cat_obese', 'bmi_cat_overweight', 'bmi_cat_underweight', 'bmi_cat_unknown', 'imd_affluent', 'imd_medium', 'imd_unaffluent', 'imd_unknown', 'imd_very_affluent', 'imd_very_unaffluent', 'ethnicity_asian', 'ethnicity_other', 'ethnicity_unknown', 'ethnicity_white', 'smoking_ex_smoker', 'smoking_never_smoked', 'smoking_smoker', 'smoking_unknown', 'season_autumn', 'season_spring', 'season_summer', 'season_winter', 'region_east', 'region_east_midlands', 'region_london', 'region_north_east', 'region_north_west', 'region_south_east', 'region_south_west', 'region_west_midlands', 'region_yorkshire', 'antibacterial_brit', 'ab_binary', 'period_during_pandemic', 'period_post_2nd_lockdown', 'period_prepandemic'])
    data = data.sort_values('variable')
    #keep mean values of gp_count and antibacterial_brit
    data['mean'] = np.round(data['mean'], decimals=2)
    #reset index to prevent issue in calculating total value
    data = data.reset_index(drop=True)
    #calculate total from sex_male and sex_female
    total = data.iloc[data[data['variable']=='sex_female'].index[0],data.columns.get_loc('count')] + data.iloc[data[data['variable']=='sex_male'].index[0],data.columns.get_loc('count')]
    #create a new dataframe for the new row and add comma to the total's thousands
    new_row = pd.DataFrame({'variable':'Total', 'count':'-', 'mean':'-', 'std':'-', 'count_int':'-', 'mean_perc':'-', 'count_mean':total}, index=[0])
    new_row['count_mean'] = new_row['count_mean'].astype(int)
    new_row['count_mean'] = new_row['count_mean'].apply(lambda x : "{:,}".format(x))
    #keep 2 digits of mean_perc
    data['mean_perc'] = data['mean_perc'].apply(lambda x: "{:.2f}".format(x))
    #add comma after 3 digits
    data['count_int'] = data['count_int'].astype(int)
    data['count_int'] = data['count_int'].apply(lambda x : "{:,}".format(x))
    #replace SN
    for row in range(0, len(data)):
        if data['count'][row] == 5:
            data['count_int'][row] = 'SN'
            data['mean_perc'][row] = 'SN'
    #combine counts and means
    data['count_mean'] = data['count_int'].astype(str) + ' (' + data['mean_perc'].astype(str) + ')'
    #append row to the dataframe
    data = pd.concat([data, new_row])
    #replace
    data['std_round'] = pd.to_numeric(data['std'],errors='coerce')
    data['std_round'] = np.round(data['std_round'], decimals=2)
    data.iloc[int(data[data['variable']=='antibacterial_brit'].index[0]),int(data.columns.get_loc('count_mean'))] = str(data.iloc[int(data[data['variable']=='antibacterial_brit'].index[0]),int(data.columns.get_loc('mean'))]) + ' (' + str(data.iloc[int(data[data['variable']=='antibacterial_brit'].index[0]),int(data.columns.get_loc('std_round'))]) + ')'

    return data

## 1- process data to make basline table, and save it

In [6]:
summary_table_lrti_incdt = data_process(summary_table_lrti_incdt)
summary_table_lrti_prevl = data_process(summary_table_lrti_prevl)
summary_table_urti_incdt = data_process(summary_table_urti_incdt)
summary_table_urti_prevl = data_process(summary_table_urti_prevl)
summary_table_uti_incdt = data_process(summary_table_uti_incdt)
summary_table_uti_prevl = data_process(summary_table_uti_prevl)
summary_table_sinusitis_incdt = data_process(summary_table_sinusitis_incdt)
summary_table_sinusitis_prevl = data_process(summary_table_sinusitis_prevl)
summary_table_otmedia_incdt = data_process(summary_table_otmedia_incdt)
summary_table_otmedia_prevl = data_process(summary_table_otmedia_prevl)
summary_table_ot_externa_incdt = data_process(summary_table_ot_externa_incdt)
summary_table_ot_externa_prevl = data_process(summary_table_ot_externa_prevl)
summary_table_urti_combined_incdt = data_process(summary_table_urti_combined_incdt)
summary_table_urti_combined_prevl = data_process(summary_table_urti_combined_prevl)
summary_table_cough_incdt = data_process(summary_table_cough_incdt)
summary_table_cough_prevl = data_process(summary_table_cough_prevl)
summary_table_cough_cold_incdt = data_process(summary_table_cough_cold_incdt)
summary_table_cough_cold_prevl = data_process(summary_table_cough_cold_prevl)
summary_table_throat_incdt = data_process(summary_table_throat_incdt)
summary_table_throat_prevl = data_process(summary_table_throat_prevl)

In [7]:
summary_table_throat_prevl

,variable,count,mean,std,count_int,mean_perc,count_mean,std_round
0,age_cat_15_24,2485.0,0.14,0.348428,"2,485",14.14,"2,485 (14.14)",0.35
1,age_cat_25_34,2430.0,0.14,0.345235,"2,430",13.83,"2,430 (13.83)",0.35
2,age_cat_35_44,2505.0,0.14,0.349829,"2,505",14.28,"2,505 (14.28)",0.35
3,age_cat_45_54,2480.0,0.14,0.348369,"2,480",14.13,"2,480 (14.13)",0.35
4,age_cat_55_64,2525.0,0.14,0.350815,"2,525",14.37,"2,525 (14.37)",0.35
5,age_cat_65_74,2560.0,0.15,0.35277,"2,560",14.57,"2,560 (14.57)",0.35
6,age_cat_75_more,2580.0,0.15,0.353968,"2,580",14.69,"2,580 (14.69)",0.35
7,sex_female,8955.0,0.51,0.499917,"8,955",50.99,"8,955 (50.99)",0.50
8,sex_male,8610.0,0.49,0.499917,"8,610",49.01,"8,610 (49.01)",0.50
9,CCI_cat_high,1805.0,0.1,0.303604,"1,805",10.27,"1,805 (10.27)",0.30


In [8]:
d = {'variable': summary_table_lrti_incdt['variable'],
     
     'lrti_incdt': summary_table_lrti_incdt['count_mean'],
     'lrti_prevl': summary_table_lrti_prevl['count_mean'],

     'urti_combined_incdt': summary_table_urti_combined_incdt['count_mean'],
     'urti_combined_prevl': summary_table_urti_combined_prevl['count_mean'],
     
     'uti_incdt': summary_table_uti_incdt['count_mean'],
     'uti_prevl': summary_table_uti_prevl['count_mean'],
     
     'sinusitis_incdt': summary_table_sinusitis_incdt['count_mean'],
     'sinusitis_prevl': summary_table_sinusitis_prevl['count_mean'],
     
     'otmedia_incdt': summary_table_otmedia_incdt['count_mean'],
     'otmedia_prevl': summary_table_otmedia_prevl['count_mean'],
     
     'ot_externa_incdt': summary_table_ot_externa_incdt['count_mean'],
     'ot_externa_prevl': summary_table_ot_externa_prevl['count_mean'],   
     
     'urti_incdt': summary_table_urti_incdt['count_mean'],
     'urti_prevl': summary_table_urti_prevl['count_mean'],

     'cough_incdt': summary_table_cough_incdt['count_mean'],
     'cough_prevl': summary_table_cough_prevl['count_mean'],

     'cough_cold_incdt': summary_table_cough_cold_incdt['count_mean'],
     'cough_cold_prevl': summary_table_cough_cold_prevl['count_mean'],

     'throat_incdt': summary_table_throat_incdt['count_mean'],
     'throat_prevl': summary_table_throat_prevl['count_mean'],
    }
data = pd.DataFrame(data=d)

data.to_csv(f'../output/aggregate_tables/baseline_table_all_infections.csv', index=False)
data

,variable,lrti_incdt,lrti_prevl,urti_combined_incdt,urti_combined_prevl,uti_incdt,uti_prevl,sinusitis_incdt,sinusitis_prevl,otmedia_incdt,...,ot_externa_incdt,ot_externa_prevl,urti_incdt,urti_prevl,cough_incdt,cough_prevl,cough_cold_incdt,cough_cold_prevl,throat_incdt,throat_prevl
0,age_cat_15_24,"23,070 (14.28)","2,570 (14.13)","90,745 (14.20)","10,120 (14.20)","23,030 (14.30)","2,560 (14.21)","22,640 (14.13)","2,580 (14.28)","23,095 (14.25)",...,"23,405 (14.58)","2,625 (14.70)","22,635 (14.21)","2,560 (14.22)","22,675 (14.19)","2,530 (14.24)","22,905 (14.26)","2,545 (14.20)","22,530 (14.14)","2,485 (14.14)"
1,age_cat_25_34,"22,880 (14.16)","2,545 (13.98)","92,285 (14.44)","10,265 (14.40)","22,860 (14.20)","2,580 (14.34)","22,845 (14.26)","2,530 (14.01)","23,060 (14.23)",...,"22,640 (14.11)","2,510 (14.06)","23,050 (14.47)","2,610 (14.49)","23,075 (14.44)","2,635 (14.81)","23,230 (14.46)","2,595 (14.47)","22,930 (14.39)","2,430 (13.83)"
2,age_cat_35_44,"23,080 (14.28)","2,525 (13.87)","90,455 (14.15)","10,185 (14.29)","23,275 (14.46)","2,505 (13.92)","22,985 (14.34)","2,700 (14.93)","23,260 (14.35)",...,"22,910 (14.28)","2,530 (14.17)","22,555 (14.16)","2,565 (14.25)","22,560 (14.12)","2,590 (14.57)","22,875 (14.24)","2,525 (14.08)","22,465 (14.10)","2,505 (14.28)"
3,age_cat_45_54,"22,990 (14.23)","2,610 (14.34)","91,475 (14.31)","10,030 (14.07)","23,190 (14.40)","2,610 (14.49)","22,895 (14.29)","2,555 (14.13)","23,280 (14.37)",...,"22,870 (14.25)","2,530 (14.16)","22,790 (14.30)","2,540 (14.10)","22,670 (14.18)","2,495 (14.02)","23,110 (14.38)","2,515 (14.02)","22,905 (14.38)","2,480 (14.13)"
4,age_cat_55_64,"23,495 (14.54)","2,635 (14.49)","91,355 (14.29)","10,210 (14.32)","22,570 (14.02)","2,590 (14.40)","23,150 (14.45)","2,645 (14.64)","23,310 (14.38)",...,"22,925 (14.29)","2,505 (14.03)","22,720 (14.26)","2,575 (14.29)","22,835 (14.29)","2,500 (14.06)","23,015 (14.33)","2,610 (14.57)","22,780 (14.30)","2,525 (14.37)"
5,age_cat_65_74,"23,055 (14.27)","2,680 (14.72)","91,875 (14.37)","10,255 (14.38)","23,175 (14.39)","2,560 (14.21)","23,020 (14.37)","2,505 (13.86)","22,825 (14.09)",...,"22,525 (14.04)","2,555 (14.30)","23,300 (14.62)","2,620 (14.55)","22,865 (14.30)","2,540 (14.29)","22,935 (14.27)","2,535 (14.14)","22,780 (14.30)","2,560 (14.57)"
6,age_cat_75_more,"23,050 (14.26)","2,630 (14.46)","90,940 (14.23)","10,215 (14.33)","22,910 (14.23)","2,600 (14.44)","22,705 (14.17)","2,560 (14.16)","23,220 (14.33)",...,"23,200 (14.46)","2,600 (14.56)","22,265 (13.98)","2,540 (14.10)","23,150 (14.48)","2,490 (14.01)","22,585 (14.06)","2,600 (14.52)","22,940 (14.40)","2,580 (14.69)"
7,sex_female,"82,555 (51.08)","9,135 (50.23)","324,945 (50.84)","36,495 (51.20)","81,890 (50.86)","9,205 (51.13)","81,805 (51.05)","9,275 (51.31)","82,750 (51.06)",...,"81,970 (51.08)","9,140 (51.18)","80,845 (50.74)","9,220 (51.19)","81,920 (51.26)","9,170 (51.56)","81,515 (50.74)","9,155 (51.08)","80,670 (50.63)","8,955 (50.99)"
8,sex_male,"79,060 (48.92)","9,055 (49.77)","314,180 (49.16)","34,785 (48.80)","79,125 (49.14)","8,800 (48.87)","78,430 (48.95)","8,800 (48.69)","79,305 (48.94)",...,"78,505 (48.92)","8,720 (48.82)","78,470 (49.26)","8,790 (48.81)","77,910 (48.74)","8,615 (48.44)","79,140 (49.26)","8,765 (48.92)","78,660 (49.37)","8,610 (49.01)"
9,CCI_cat_high,"16,920 (10.47)","1,875 (10.30)","67,535 (10.57)","7,415 (10.40)","17,010 (10.56)","1,960 (10.88)","16,630 (10.38)","1,855 (10.25)","16,695 (10.30)",...,"16,930 (10.55)","1,870 (10.48)","16,745 (10.51)","1,885 (10.45)","16,695 (10.44)","1,845 (10.38)","17,100 (10.64)","1,880 (10.49)","16,995 (10.67)","1,805 (10.27)"
